In [ ]:
import os
import glob
import datetime
import numpy as np
import tensorflow as tf

In [ ]:
%load_ext tensorboard
!rm -rf ./logs/

In [ ]:
from keras.preprocessing.image import img_to_array, load_img
from keras.utils import to_categorical

In [ ]:
train_files = glob.glob('train_data/*')
train_imgs = np.array([img_to_array(load_img(img, target_size=(256,256,3))) for img in train_files])
train_labels = to_categorical([int(float(filename.split('\\')[1].strip('.jpg'))) for filename in train_files])

val_files = glob.glob('val_data/*')
val_imgs = np.array([img_to_array(load_img(img, target_size=(256,256,3))) for img in val_files])
val_labels = to_categorical([int(float(filename.split('\\')[1].strip('.jpg'))) for filename in val_files])

test_files = glob.glob('test_data/*')
test_imgs = np.array([img_to_array(load_img(img, target_size=(256,256,3))) for img in test_files])
test_labels = to_categorical([int(float(filename.split('\\')[1].strip('.jpg'))) for filename in test_files])

In [ ]:
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_generator = ImageDataGenerator(rescale=1./255, zoom_range=0.3, rotation_range=50,
                              width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, 
                              horizontal_flip=True, fill_mode='nearest')

val_generator = ImageDataGenerator(rescale=1./255)

train_gen = train_generator.flow(train_imgs, train_labels, batch_size=2)
val_gen = val_generator.flow(val_imgs, val_labels, batch_size=2)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
checkpoint = ModelCheckpoint('final_model.h5', save_best_only=True)
earlystoppage = EarlyStopping(patience=3, restore_best_weights=True)
callbacks = [earlystoppage, checkpoint]

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dropout, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.applications.xception import Xception, preprocess_input, decode_predictions

In [ ]:
log_dir = os.path.join("logs", "fit", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#### Transfer Learning
- Froze all layers in model to forego retraining everything
- Added two layers onto it - first as another dense layer and the next as the image classifier

In [ ]:
base = Xception(include_top = False, weights='imagenet', input_shape = (256, 256, 3))
for layers in base.layers[:]:
    layers.trainable = False

In [ ]:
xception_architecture = base.output
pool = GlobalAveragePooling2D()(xception_architecture)
dense = Dense(1024, activation='relu')(pool)
predictions = Dense(114, activation='softmax')(dense) 
model = Model(inputs=base.input, outputs=predictions)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit(train_gen, epochs = 5, verbose=1, validation_data = val_gen, shuffle = True, callbacks = [tensorboard_callback])

train_loss, train_acc = model.evaluate(train_imgs, train_labels)
test_loss, test_acc = model.evaluate(test_imgs, test_labels)
print('Train: %.3f, Test: %.3f' % (train_acc * 100, test_acc * 100))

In [ ]:
%tensorboard --logdir logs/fit

#### Predictions
- Loaded in three pictures; the model only been trained on the first one
- Predicted the final result

In [ ]:
img_path_ethan = 'preds/ethan.jpg'
img_path_andrew = 'preds/andrew.jpg'
img_path_nikki = 'preds/nikki.jpg'

img1 = image.load_img(img_path_ethan, target_size = (256, 256))
img2 = image.load_img(img_path_andrew, target_size = (256, 256))
img3 = image.load_img(img_path_nikki, target_size = (256, 256))

In [ ]:
img1_pred = image.img_to_array(img1)
img1_pred = np.expand_dims(img1_pred, axis=0)
img1_pred = preprocess_input(img1_pred)

img2_pred = image.img_to_array(img2)
img2_pred = np.expand_dims(img2_pred, axis=0)
img2_pred = preprocess_input(img2_pred)

img3_pred = image.img_to_array(img3)
img3_pred = np.expand_dims(img3_pred, axis=0)
img3_pred = preprocess_input(img3_pred)

In [ ]:
arr = model.predict(img1_pred)

In [ ]:
result = np.where(arr == np.amin(arr))
print('List of Indices of maximum element :', result)

In [ ]:
model.save('final_model.h5')